In [ ]:
from dask.distributed import Client

In [ ]:
client = Client()

In [ ]:
client

In [ ]:
def square(x):
    return x**2

In [ ]:
def neg(x):
    return -x

In [ ]:
A = client.map(square, range(100000))

In [ ]:
B = client.map(neg, A)

In [ ]:
total = client.submit(sum, B)

In [ ]:
total

In [ ]:
total.result()